Hate Speech and Explicit Content Detection

 # <----- Data Pre-processing ----->

dataset: https://www.kaggle.com/datasets/waalbannyantudre/hate-speech-detection-curated-dataset

class label for majority of CF users. “0″ means non-hateful
“1″ means hateful

In [35]:
# Imports

import pandas as pd
import numpy as np
import random

In [36]:
# More imports

import textacy
import textacy.preprocessing.normalize as tprep
import unidecode
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [37]:
# Imports

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
# <-- RUN ONCE -->

#nltk.download('stopwords')
#nltk.download("wordnet")
#nltk.download("onw-1.4")
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\jakeh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\jakeh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\jakeh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\jakeh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\jakeh\AppData\Roaming\nltk_data...
[

True

In [39]:
# Importing dataset

file = "Datasets/Dataset2/archive/HateSpeechDataset.csv"

# Counting number of rows in dataset
n = sum(1 for line in open(file))-1

# Getting a sample size of 5%
s = n//5

# Skip rows
skip = sorted(random.sample(range(1, n+1), n-s))

df_load = pd.read_csv(file)

df_load.shape

(440906, 3)

In [40]:
df_load.head(10)

,Content,Label,Content_int
0,denial of normal the con be asked to comment o...,1,"[146715, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,..."
1,just by being able to tweet this insufferable ...,1,"[146715, 14, 15, 16, 17, 7, 18, 19, 20, 21, 22..."
2,that is retarded you too cute to be single tha...,1,"[146715, 28, 29, 30, 26, 31, 32, 7, 5, 33, 28,..."
3,thought of a real badass mongol style declarat...,1,"[146715, 35, 1, 24, 36, 37, 38, 39, 40, 1, 41,..."
4,afro american basho,1,"[146715, 46, 47, 48, 146714]"
5,yeah retard haha,1,"[146715, 49, 13, 50, 146714]"
6,the ching chong chung stuff,1,"[146715, 3, 51, 52, 53, 54, 146714]"
7,the dead what a slut still warm when she tweet...,1,"[146715, 3, 55, 56, 24, 57, 58, 59, 60, 61, 62..."
8,let your tweets be harmless it will not affect...,1,"[146715, 71, 72, 73, 5, 74, 63, 75, 76, 77, 78..."
9,these latinos who have a problem with immigrat...,1,"[146715, 85, 86, 67, 87, 24, 88, 89, 90, 91, 9..."


In [41]:
# Changing dataset classes to match dataset 1

df_load['Label'].replace(0, 2, inplace=True)
df_load['Label'].replace(1, 0, inplace=True)

C:\Users\jakeh\AppData\Local\Temp\ipykernel_8788\2238973968.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_load['Label'].replace(0, 2, inplace=True)
C:\Users\jakeh\AppData\Local\Temp\ipykernel_8788\2238973968.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

# <----- Dataset Optimisation ----->

In [42]:
df_load.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440906 entries, 0 to 440905
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Content      440906 non-null  object
 1   Label        440906 non-null  object
 2   Content_int  440906 non-null  object
dtypes: object(3)
memory usage: 10.1+ MB


In [43]:
# Missing values

df_load.isnull().any()

Content        False
Label          False
Content_int    False
dtype: bool

In [44]:
# Duplicate Values

duplicated_rows_df = df_load[df_load.duplicated()]
print("No of dup rows: ", duplicated_rows_df.shape)
df = df_load.drop_duplicates()

No of dup rows:  (22378, 3)


# <----- Dataset Textual Analysis Prep ----->

In [45]:
# Data Normalization and cleaning

df.head(1)

tweet = df['Content'].to_numpy()

tweet2 = []

for x in tweet:
    x = x.lower()
    x = tprep.hyphenated_words(x)
    x = tprep.quotation_marks(x)
    x = tprep.unicode(x)
    x = unidecode.unidecode(x)
    x = re.sub(r'[^\w\s]', '', x)

    tweet2.append(x)

df_tweet = pd.Series(tweet2)

df_tweet.head(20)


0     denial of normal the con be asked to comment o...
1     just by being able to tweet this insufferable ...
2     that is retarded you too cute to be single tha...
3     thought of a real badass mongol style declarat...
4                                   afro american basho
5                                      yeah retard haha
6                           the ching chong chung stuff
7     the dead what a slut still warm when she tweet...
8     let your tweets be harmless it will not affect...
9     these latinos who have a problem with immigrat...
10    i feel so much secondhand embarrassment when a...
11    you have got a gorgeous figure what an unfunny...
12                                   what a vile vagina
13                                  oh shut up you twat
14    because i can you fucking retard what does up ...
15                                  this fucking faggot
16          holy shit what a faggot laughing my ass off
17    how you on hate on man laughing my ass off

In [46]:
# Stopword Removal

stop_words = set(stopwords.words('english'))

df_tweets2 = df_tweet.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

df_tweets2.shape

(418528,)

In [47]:
df_tweets2.head(5)


0    denial normal con asked comment tragedies emot...
1    able tweet insufferable bullshit proves trump ...
2                            retarded cute single life
3    thought real badass mongol style declaration w...
4                                  afro american basho
dtype: object

In [48]:
# Lemmatization

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()

lemm_data = df_tweets2.to_numpy()

for x in lemm_data:
    str = x.split()
    for x in str:
        x = lemmatizer.lemmatize(x)

df_tweets3 = pd.Series(lemm_data)

df_tweets3.head(10)


0    denial normal con asked comment tragedies emot...
1    able tweet insufferable bullshit proves trump ...
2                            retarded cute single life
3    thought real badass mongol style declaration w...
4                                  afro american basho
5                                     yeah retard haha
6                              ching chong chung stuff
7    dead slut still warm tweeted slut vagina mate ...
8          let tweets harmless affect way faggot one n
9    latinos problem immigration enforcement stay s...
dtype: object

# <----- Teaxtaul Analysis ----->

In [49]:
# Dataset Values

final_tweets = df_tweets3
value =  df['Label']

processed_df = pd.DataFrame({'Content': final_tweets, 'value': value})

processed_df.head(78)


,Content,value
0,denial normal con asked comment tragedies emot...,1
1,able tweet insufferable bullshit proves trump ...,1
2,retarded cute single life,1
3,thought real badass mongol style declaration w...,1
4,afro american basho,1
...,...,...
73,london fans hate fat vagina means times office...,1
74,call singing vagina would get four performance...,1
75,need cool cowhide coat extraordinary event dwa...,1
76,ching chong bing bong eat dog,1


In [50]:
# Word Count

processed_df['word_count'] = processed_df['Content'].str.count(' ') + 1

processed_df.head(1)



,Content,value,word_count
0,denial normal con asked comment tragedies emot...,1,8.0


In [51]:
# Explicit Content

# Create a sep model to detect explicit content



In [53]:
# Sentiment Analysis

analyser = SentimentIntensityAnalyzer()

def getSentiment(text):
    scores = analyser.polarity_scores(text)
    sentiment = 1 if scores['pos'] > 0 else 0

    return sentiment

processed_df['sentiment'] = processed_df['Content'].apply(getSentiment)

sentiment = processed_df['sentiment'].to_numpy()

processed_df.head(10)

AttributeError: 'float' object has no attribute 'encode'

In [54]:
# TF-IDF Score

the_text = processed_df['Content']

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None, max_features=1000)

tfidf_tweets = tfidf.fit_transform(the_text)

vocab = tfidf.vocabulary_
feature_names = list(vocab.keys())

df_tfidf = pd.DataFrame(tfidf_tweets.toarray(), columns=feature_names)

df_tfidf.head(1)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
# Bag of words

bow = CountVectorizer(max_features=100)

bow_text_data = processed_df['Content']

#bow_text_data = bow_text_data.to_numpy()

bow.fit(bow_text_data)

bow_features = bow.get_feature_names_out()

bow_transform = bow.transform(bow_text_data)

bow_transform_array = bow_transform.toarray()

df_bow = pd.DataFrame(bow_transform_array, columns=bow_features)

df_bow.shape


(24783, 100)

In [ ]:
# final Dataset - tfidf

df_final_tfidf = df_tfidf

df_final_tfidf['orig_tweets'] = final_tweets
df_final_tfidf['sentiment_value'] = sentiment
df_final_tfidf['class_value'] = value
df_final_tfidf['word_count'] = processed_df['word_count']

df_final_tfidf.shape

(24783, 1004)

In [ ]:
# final dataset - bag of words

df_final_bow = df_bow

df_final_bow['orig_tweets'] = final_tweets
df_final_bow['sentiment_value'] = sentiment
df_final_bow['class_value'] = value

df_final_bow.shape

(24783, 103)

In [ ]:
# Export to csv file

df_final_tfidf.to_csv('dataset1Finalised2_tfidf_col_500.csv', index=False)